# pytorch 读取文件、准备数据

In [1]:
import os
import sys
import numpy as np
from tqdm import tqdm_notebook
import csv
import pandas as pd
import torch.utils.data as data
import torchvision.transforms as transforms
from PIL import Image
import torch


class SaltData(data.Dataset):

    def __init__(self):

        self.train_csv_path = "./saltdata/train.csv"
        self.path_train = "./saltdata/train"
        self.train_ids = self.get_ids(self.train_csv_path)
#         self.dataset()

    def get_ids(self, csv_path):

        # list_id中保存的是训练图片的名称，但是没有后缀
        df = pd.read_csv(csv_path)
        self.list_id = []
        for i, item in df.iterrows():
            self.list_id.append(item[0] + ".png")
        return self.list_id

    def transforms(self, im, label):

        im_tfs = transforms.Compose([transforms.Resize(96), transforms.ToTensor()])
        im = im_tfs(im)
        label = im_tfs(label)
        return im, label

#     def dataset(self):

#         print('Getting and resizing train images and masks ... ')
#         sys.stdout.flush()
#         self.imgs = []
#         self.labels = []
#         for n, id_ in tqdm_notebook(enumerate(self.train_ids), total=len(self.train_ids)):
#             img = Image.open(self.path_train + '/images/' + id_)
#             mask = Image.open(self.path_train + '/masks/' + id_)
#             img, label = self.transforms(img, mask)
#             self.imgs.append(img)
#             self.labels.append(label)
#         print('Done!')
        
    def __getitem__(self, index):
        
        id_ = self.train_ids[index]

        img = Image.open(self.path_train + '/images/' + id_).convert('RGB')
        mask = Image.open(self.path_train + '/masks/' + id_).convert('RGB')

        if self.transforms is not None:
            img, label = self.transforms(img, mask)
            label = label.type(torch.LongTensor)
        return img,  label
    
    def __len__(self):
        
        return len(self.train_ids)
    
train_dataset = SaltData()
train_loader = data.DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)

/home/loong/anaconda3/envs/pytorch3.6/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/loong/anaconda3/envs/pytorch3.6/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
from pytorch_refinenet.pytorch_refinenet import RefineNet4Cascade

print('start load model!!!')
net = RefineNet4Cascade((3, 96), num_classes=2)
print(net)
print('transfered to cuda done!!!')
optimizer = torch.optim.Adam(net.parameters(), lr= 0.1)
loss_func = torch.nn.CrossEntropyLoss()
EPOCH = 1
for epoch in range(EPOCH):
    for step, (b_x,b_y) in enumerate(train_loader):  # 分配 batch data, normalize x when iterate train_loader

        inputs = torch.autograd.Variable(b_x)
        targets = torch.autograd.Variable(b_y)
        output = net(inputs)  # VGG16 output
        loss = loss_func(output, targets).cpu()  # cross entropy loss
        optimizer.zero_grad()  # clear gradients for this training step
        loss.backward()  # backpropagation, compute gradients
        optimizer.step()  # apply gradients
        

start load model!!!
RefineNet4Cascade(
  (layer1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (downsample): Sequential(
       

/home/loong/anaconda3/envs/pytorch3.6/lib/python3.6/site-packages/torch/nn/modules/upsampling.py:122: UserWarning: nn.Upsampling is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.Upsampling is deprecated. Use nn.functional.interpolate instead.")
Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x7f94903ab908>>
Traceback (most recent call last):
  File "/home/loong/anaconda3/envs/pytorch3.6/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 399, in __del__
    self._shutdown_workers()
  File "/home/loong/anaconda3/envs/pytorch3.6/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 378, in _shutdown_workers
    self.worker_result_queue.get()
  File "/home/loong/anaconda3/envs/pytorch3.6/lib/python3.6/multiprocessing/queues.py", line 345, in get
    return _ForkingPickler.loads(res)
  File "/home/loong/anaconda3/envs/pytorch3.6/lib/python3.6/site-packages/torch/multipr

RuntimeError: invalid argument 3: only batches of spatial targets supported (3D tensors) but got targets of dimension: 4 at /opt/conda/conda-bld/pytorch_1532579805626/work/aten/src/THNN/generic/SpatialClassNLLCriterion.c:59

In [ ]:
def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)